In [1]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
import lxml # this will need to be installed via command line first(as well)
import html5lib # this will need to be installed via command line first
import numpy as np
import re

In [5]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalaryCapHits(team):
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[0]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[0].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        if len(SalariesDF) >= 51:
            SalariesDF = SalariesDF.drop(51, axis=0)
        # Add team location abv to each player
        SalariesDF = SalariesDF.assign(TeamLOC = team[1])
        SalariesDF = SalariesDF.assign(TeamName = (team[0].replace("-", " ")).title())
        
        
        # Change the order of columns to make more sense semantically 
        new_order = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        SalariesDF = SalariesDF.loc[:, new_order]
        
        # Specify filename and path
        csvPathName = f'./salary-Lists/{TeamNameCaptalized}-Player-Salary-List.csv'
        
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(csvPathName, index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'./salary-Lists/{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        print(SalariesCSV)
        
    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [6]:
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = [['buffalo-bills', 'BUF'], ['miami-dolphins', 'MIA'], ['new-england-patriots', 'NE'], ['new-york-jets', 'NYJ'],
            ['baltimore-ravens', 'BAL'], ['cincinnati-bengals', 'CIN'], ['cleveland-browns', 'CLE'], ['pittsburgh-steelers', 'PIT'],
            ['houston-texans', 'HOU'], ['indianapolis-colts', 'IND'], ['jacksonville-jaguars', 'JAX'], ['tennessee-titans', 'TEN'], 
            ['denver-broncos', 'DEN'], ['kansas-city-chiefs', 'KC'], ['las-vegas-raiders', 'LV'], ['los-angeles-chargers', 'LAC'],
            ['dallas-cowboys', 'DAL'], ['new-york-giants', 'NYG'], ['philadelphia-eagles', 'PHI'], ['washington-commanders', 'WSH'],
            ['chicago-bears', 'CHI'], ['detroit-lions', 'DET'], ['green-bay-packers', 'GB'], ['minnesota-vikings', 'MIN'],
            ['atlanta-falcons', 'ATL'], ['carolina-panthers', 'CAR'], ['new-orleans-saints', 'NO'], ['tampa-bay-buccaneers', 'TB'],
            ['arizona-cardinals', 'ARI'], ['los-angeles-rams', 'LAR'], ['san-francisco-49ers', 'SF'], ['seattle-seahawks', 'SEA']
           ]

# Call the method defined above to get all the player data
def GetMultipleTeamSalaryCapHits(TeamList):
    for team in TeamList:
        GetTeamSalaryCapHits(team)

# GetMultipleTeamSalaryCapHits(NFLTeams)
GetTeamSalaryCapHits(['san-francisco-49ers', 'SF'])

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the San Francisco 49Ers:
             Player TeamLOC             TeamName   Cap Number
0    Trent Williams      SF  San Francisco 49Ers  $31,568,542
1      Deebo Samuel      SF  San Francisco 49Ers  $28,633,765
2   Charvarius Ward      SF  San Francisco 49Ers  $18,401,000
3         Nick Bosa      SF  San Francisco 49Ers  $14,671,000
4     Brandon Aiyuk      SF  San Francisco 49Ers  $14,124,000
..              ...     ...                  ...          ...
86   Ricky Pearsall      SF  San Francisco 49Ers     $795,000
87   Tayler Hawkins      SF  San Francisco 49Ers     $795,000
88    Isaac Alarcon      SF  San Francisco 49Ers     $795,000
89     Shakel Brown      SF  San Francisco 49Ers     $795,000
90       Tay Martin      SF  San Francisco 49Ers     $795,000

[91 rows x 4 columns]
